# Week 3 assignment - test function


Contents
 - Start_. Import packages, create functions and load data
 - Test function

## Start_.

**Import packages**

In [1]:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.linear_model import ElasticNetCV, ElasticNet

import matplotlib.pyplot as plt
%matplotlib inline

**Directories and paths**

In [2]:
# Set directories
dirRawData = "../input/"
dirPData   = "../PData/"

**Load data**

In [3]:
ames_dtypes = {'Order': np.int64,
             'PID': np.int64,
             'MS.SubClass': np.object,
             'MS.Zoning': np.object,
             'Lot.Frontage': np.float64,
             'Lot.Area': np.float64,
             'Street': np.object, 
             'Alley': np.object, 
             'Lot.Shape': np.object, 
             'Land.Contour': np.object,  
             'Lot.Config': np.object,
             'Land.Slope': np.object,
             'Neighborhood': np.object,
             'Condition.1': np.object,
             'Condition.2': np.object,
             'Bldg.Type': np.object,
             'House.Style': np.object,
             'Overall.Qual': np.float64,
             'Overall.Cond': np.float64,
             'Year.Built': np.float64,
             'Year.Remod.Add': np.float64,
             'Roof.Style': np.object,
             'Roof.Matl': np.object,
             'Exterior.1st': np.object,
             'Exterior.2nd': np.object,
             'Mas.Vnr.Type': np.object,
             'Mas.Vnr.Area': np.float64,
             'Exter.Qual': np.object,
             'Exter.Cond': np.object,
             'Foundation': np.object, 
             'Bsmt.Qual': np.object,
             'Bsmt.Cond': np.object,
             'Bsmt.Exposure': np.object,
             'BsmtFin.Type.1': np.object,
             'BsmtFin.SF.1': np.float64,
             'BsmtFin.Type.2': np.object,
             'BsmtFin.SF.2': np.float64,
             'Bsmt.Unf.SF': np.float64,
             'Total.Bsmt.SF': np.float64,
             'Heating': np.object,
             'Heating.QC': np.object,
             'Central.Air': np.object,
             'Electrical': np.object,
             'X1st.Flr.SF': np.float64,
             'X2nd.Flr.SF': np.float64,
             'Low.Qual.Fin.SF': np.float64,
             'Gr.Liv.Area': np.float64, 
             'Bsmt.Full.Bath': np.float64,
             'Bsmt.Half.Bath': np.float64,
             'Full.Bath': np.float64,
             'Half.Bath': np.float64,
             'Bedroom.AbvGr': np.float64,
             'Kitchen.AbvGr': np.float64,
             'Kitchen.Qual': np.object,
             'TotRms.AbvGrd': np.float64, 
             'Functional': np.object, 
             'Fireplaces': np.float64, 
             'Fireplace.Qu': np.object,
             'Garage.Type': np.object, 
             'Garage.Yr.Blt': np.float64,
             'Garage.Finish': np.object,
             'Garage.Cars': np.float64,
             'Garage.Area': np.float64, 
             'Garage.Qual': np.object, 
             'Garage.Cond': np.object, 
             'Paved.Drive': np.object,
             'Wood.Deck.SF': np.float64,
             'Open.Porch.SF': np.float64,
             'Enclosed.Porch': np.float64, 
             'X3Ssn.Porch': np.float64,
             'Screen.Porch': np.float64,
             'Pool.Area': np.float64,
             'Fence': np.object,
             'Misc.Feature': np.object,
             'Misc.Val': np.float64,
             'Mo.Sold': np.float64, 
             'Yr.Sold': np.float64,
             'Sale.Type': np.object,
             'Sale.Condition': np.object,
             'SalePrice': np.float64}


In [4]:
df_all = pd.read_csv(dirRawData + 'AmesHousing.txt', 
                     sep=" ",
                     dtype = ames_dtypes,
                     na_values = 'NA')

### Test function

In [5]:
# import your function (replace studentnumber" !!)
from n_190011697 import fn_ames_en

In [6]:
# create a copy of the data for testing
df_all_testingcopy = df_all.copy()

Call your function

In [7]:
en_, X, y, mae_design, mae_test = fn_ames_en(df_all_testingcopy)

Some extra data prep so you can test your function

In [8]:
# now create folds and indices to carry out tests
import re
    
def convert(name):
    s1 = re.sub('\.', '_', name)
    return s1.lower()

# change column names to closer to camel case
colnames = df_all.columns.values
colnames = list(map(convert, colnames))
df_all.columns = colnames
del convert, colnames
    
# Deal with missings as per 02a
vars_toDrop = ['lot_frontage', 'garage_yr_blt', 'mas_vnr_area']
df_all.drop(labels=vars_toDrop,
            axis=1,
            inplace=True)
    
df_all.dropna(inplace = True)

# remove outliers
df_all = df_all[df_all['gr_liv_area'] <= 4000]
df_all.reset_index(drop=True, inplace=True)
    
# create fold
rng = np.random.RandomState(2018)
fold = rng.randint(0, 10, df_all.shape[0])
df_all['fold'] = fold

# define index for train, val, design, test
idx_train  = np.where(df_all['fold'].isin(np.arange(0,6)))[0]
idx_val    = np.where(df_all['fold'].isin([6,7]))[0]
idx_design = np.where(df_all['fold'].isin(np.arange(0,8)))[0]
idx_test   = np.where(df_all['fold'].isin([8,9]))[0]

print(len(idx_train))  # 1,749
print(len(idx_val))    #   586
print(len(idx_design)) # 2,335

1749
586
2335


In [9]:
def fn_MAE(actuals, predictions):
    return np.round(np.mean(np.abs(predictions - actuals)), 0)

Finaly test the function 

In [10]:
X_design = X[idx_design, :]
X_test   = X[idx_test, :]
y_design = df_all['saleprice'].iloc[idx_design].copy().values.ravel()
y_test   = df_all['saleprice'].iloc[idx_test].copy().values.ravel()

pred_design = en_.predict(X_design)
pred_test   = en_.predict(X_test)

# calculate MAE on test and non test but then hard code in the return statement
mae_design_my_calc = fn_MAE(y_design, pred_design)
mae_test_my_calc   = fn_MAE(y_test, pred_test)


In [11]:
mae_design_my_calc == mae_design

True

In [12]:
mae_test_my_calc == mae_test

True

In [13]:
mae_test<13700

True

In [14]:
mae_test-mae_design<1100

True

In [15]:
mae_test

13655

In [17]:
mae_test-mae_design

1026